In [16]:
import pandas as pd

file_path = r"C:\Users\User\OneDrive\桌面\My Course\碩一下\給台大資料\Data1-unfiltered.xlsx"  


data1_df = pd.read_excel(file_path)  
data1_df = data1_df[data1_df["車種"].str.contains("1.9T|電動車", na=False) | (data1_df["車種"] == "機車")]

invalid_staff = ["人派", "人派1", "已取貨"]
data1_df = data1_df[~data1_df["作業人員"].isin(invalid_staff)]

data1_df["車種"] = data1_df["車種"].replace({
    "1.9T常溫": "1.9T",
    "1.9T低溫": "1.9T"
})
data1_df["三邊加總 (CM)"] = data1_df["長"] + data1_df["寬"] + data1_df["高"]

file_path2 = r"C:\Users\User\OneDrive\桌面\My Course\碩一下\給台大資料\Data2.xlsx" 

data2_df = pd.read_excel(file_path2)
data2_df["購車金額"] = data2_df["購車金額"] / 5 / 12
data2_df.rename(columns={"購車金額": "月折舊費用 (Monthly Depreciation)"}, inplace=True)

print(data1_df.head())
print(data2_df.head())


            配送單號  作業內容                作業時間 作業人員      營業所        路線代碼 當配/一般  \
2   300004241261   運送中 2024-12-11 09:30:40  趙祺弘  NS16-欣華  NS16-01-22    當配   
3   300004241261  完成運送 2024-12-11 20:09:03  趙祺弘  NS16-欣華  NS16-01-22    當配   
6   300004241272   運送中 2024-12-11 09:31:26  趙祺弘  NS16-欣華  NS16-01-22    當配   
7   300004241272  完成運送 2024-12-11 20:08:51  趙祺弘  NS16-欣華  NS16-01-22    當配   
10  300004242436   運送中 2024-12-12 08:57:32  謝高曉  NS16-欣華  NS16-01-32    一般   

       長     寬     高   箱號   箱型  車種   重量  三邊加總 (CM)  
2   45.0  30.0  30.0  NaN  NaN  機車  NaN      105.0  
3   45.0  30.0  30.0  NaN  NaN  機車  NaN      105.0  
6   90.0  20.0  35.0  NaN  NaN  機車  NaN      145.0  
7   90.0  20.0  35.0  NaN  NaN  機車  NaN      145.0  
10  45.0  30.0  30.0  NaN  1.0  機車  1.0      105.0  
     車種  月折舊費用 (Monthly Depreciation)  油資/電費 ($/配送件數)  駕駛人員月薪  人員獎金($/配送件數) 
0  1.9T                   9500.000000            2.97   37000           12.6
1    機車                   1475.000000            0.60   310

In [24]:
# Step 1: Remove rows with missing required fields
x_df = data1_df.dropna(subset=["配送單號", "作業人員", "車種"])

# Step 2: Drop duplicates to count only unique delivery assignments
x_unique = x_df.drop_duplicates(subset=["配送單號", "作業人員", "車種"])

# Step 3: Calculate total deliveries per month across all vehicle types
total_deliveries = len(x_unique) / 2.5

print("Estimated total monthly deliveries:", total_deliveries)



Estimated total monthly deliveries: 54213.2


In [28]:
# Step 1: Fill missing "三邊加總 (CM)" based on 車種 rules
cm_filled = data1_df.copy()

cm_missing = cm_filled["三邊加總 (CM)"].isna()
cm_filled.loc[cm_missing & (cm_filled["車種"] == "機車"), "三邊加總 (CM)"] = 75
cm_filled.loc[cm_missing & (cm_filled["車種"] == "電動車常溫"), "三邊加總 (CM)"] = 105
cm_filled.loc[cm_missing & (cm_filled["車種"] == "1.9T"), "三邊加總 (CM)"] = 121  # forces to be treated as "large"
valid_df = cm_filled.dropna(subset=["配送單號", "作業人員", "車種", "三邊加總 (CM)"])

large_df = valid_df[valid_df["三邊加總 (CM)"] > 120]

large_unique = large_df.drop_duplicates(subset=["配送單號", "作業人員", "車種"])

large_delivery_count = len(large_unique)
monthly_large_delivery = large_delivery_count / 2.5

print(f"Total unique large-item assignments: {large_delivery_count}")
print(f"Estimated large-item deliveries per month: {monthly_large_delivery:.2f}")



Total unique large-item assignments: 9659
Estimated large-item deliveries per month: 3863.60


In [27]:
# Use CM-filled data from earlier
cm_filled = data1_df.copy()

# Fill missing CM as before (if not already filled)
cm_missing = cm_filled["三邊加總 (CM)"].isna()
cm_filled.loc[cm_missing & (cm_filled["車種"] == "機車"), "三邊加總 (CM)"] = 75
cm_filled.loc[cm_missing & (cm_filled["車種"] == "電動車常溫"), "三邊加總 (CM)"] = 105
cm_filled.loc[cm_missing & (cm_filled["車種"] == "1.9T"), "三邊加總 (CM)"] = 121
valid_df = cm_filled.dropna(subset=["配送單號", "作業人員", "車種", "三邊加總 (CM)"])

# Drop duplicates
delivery_df = valid_df.drop_duplicates(subset=["配送單號", "作業人員", "車種"])

# Eligible for 機車 (CM < 90)
moto_eligible = delivery_df[delivery_df["三邊加總 (CM)"] <= 90]
moto_monthly_max = len(moto_eligible) / 2.5

# Eligible for 電動機車 (CM < 120)
emoto_eligible = delivery_df[delivery_df["三邊加總 (CM)"] <= 120]
emoto_monthly_max = len(emoto_eligible) / 2.5

print(f"Max deliveries for 機車 (CM < 90): {moto_monthly_max:.2f}")
print(f"Max deliveries for 電動機車 (CM < 120): {emoto_monthly_max:.2f}")


Max deliveries for 機車 (CM < 90): 43592.00
Max deliveries for 電動機車 (CM < 120): 50349.60


In [52]:
###abandoned###

# Step 1: Ensure 作業時間 is datetime and extract date
data1_df["作業時間"] = pd.to_datetime(data1_df["作業時間"], errors="coerce")
data1_df["作業日期"] = data1_df["作業時間"].dt.date

# Step 2: Filter valid rows
valid = data1_df.dropna(subset=["配送單號", "作業人員", "車種", "作業日期"])

# Step 3: Define trip: 作業人員 + 車種 + 作業日期
trip_df = valid.drop_duplicates(subset=["配送單號", "作業人員", "車種", "作業日期"])

# Step 4: Count deliveries per trip
trip_deliveries = trip_df.groupby(["作業人員", "車種", "作業日期"])["配送單號"].nunique().reset_index(name="delivery_count")

# Step 5: Calculate average deliveries per trip by 車種
avg_deliveries_per_trip = trip_deliveries.groupby("車種")["delivery_count"].mean().reset_index(name="avg_deliveries_per_trip")

# Step 6: Count trips per 車種
trip_counts = trip_deliveries["車種"].value_counts().reset_index()
trip_counts.columns = ["車種", "trip_count"]
trip_counts["trips_per_month"] = trip_counts["trip_count"] / 2.5

# Step 7: Merge to calculate delivery capacity per vehicle per month
vehicle_capacity = pd.merge(avg_deliveries_per_trip, trip_counts, on="車種")
vehicle_capacity["deliveries_per_vehicle_per_month"] = vehicle_capacity["avg_deliveries_per_trip"] * vehicle_capacity["trips_per_month"]

# Display results
print(vehicle_capacity)


      車種  avg_deliveries_per_trip  trip_count  trips_per_month  \
0   1.9T                66.227523         545            218.0   
1     機車                52.801489        1209            483.6   
2  電動車常溫               120.797468         316            126.4   

   deliveries_per_vehicle_per_month  
0                           14437.6  
1                           25534.8  
2                           15268.8  


In [43]:
import pulp

# Step 1: Define the LP minimization problem
model = pulp.LpProblem("Vehicle_Import_Optimization", pulp.LpMinimize)

# Step 2: Decision Variables
y_1_9t = pulp.LpVariable("y_1.9T", lowBound=0, cat="Integer")
y_emoto = pulp.LpVariable("y_電動機車", lowBound=0, cat="Integer")
y_moto = pulp.LpVariable("y_機車", lowBound=0, cat="Integer")

x_1_9t = pulp.LpVariable("x_1.9T", lowBound=0)
x_emoto = pulp.LpVariable("x_電動機車", lowBound=0)
x_moto = pulp.LpVariable("x_機車", lowBound=0)

# Step 3: Constraints

# 1. Total delivery demand
model += x_1_9t + x_emoto + x_moto >= 54213.2, "Total_Monthly_Delivery_Demand"

# 2. Large items must go to 1.9T
model += x_1_9t >= 3863.6, "Large_Item_Only_By_1.9T"

# 3. Size constraints
model += x_moto <= 43592.0, "Motorcycle_Size_Limit"
model += x_emoto <= 50349.6, "EMoto_Size_Limit"

# Step 4: Objective Function (Fixed + Variable costs)
model += (
    y_1_9t * (9500 + 37000) + x_1_9t * (2.97 + 12.6) +
    y_emoto * (8274.92 + 31000) + x_emoto * (0.37 + 12.2) +
    y_moto * (1475 + 31000) + x_moto * (0.6 + 12.2)
), "Total_Monthly_Cost"

model += x_moto <= y_moto * 14437.6 
model += x_emoto <= y_emoto * 25534.8 
model += x_1_9t <= y_1_9t * 15268.8 

# Step 5: Solve the model
model.solve()

# Step 6: Output the results
print("Status:", pulp.LpStatus[model.status])

print("\nOptimal number of vehicles to import:")
print(f"1.9T: {y_1_9t.varValue}")
print(f"電動機車: {y_emoto.varValue}")
print(f"機車: {y_moto.varValue}")

print("\nDeliveries assigned per month:")
print(f"1.9T: {x_1_9t.varValue}")
print(f"電動機車: {x_emoto.varValue}")
print(f"機車: {x_moto.varValue}")

print("\nTotal Estimated Monthly Cost:", pulp.value(model.objective))


Status: Optimal

Optimal number of vehicles to import:
1.9T: 1.0
電動機車: 2.0
機車: 0.0

Deliveries assigned per month:
1.9T: 3863.6
電動機車: 50349.6
機車: 0.0

Total Estimated Monthly Cost: 818100.564


In [42]:
# Make sure 作業時間 is datetime
data1_df["作業時間"] = pd.to_datetime(data1_df["作業時間"], errors="coerce")
data1_df["作業日期"] = data1_df["作業時間"].dt.date

# Drop bad data (NaNs and invalid operators)
valid_fleet_df = data1_df.dropna(subset=["車種", "作業人員", "作業日期"])

# One row per (driver, vehicle type, date) = one vehicle used that day
daily_vehicles = valid_fleet_df.drop_duplicates(subset=["作業人員", "車種", "作業日期"])

# Count vehicles used per day per 車種
daily_vehicle_use = daily_vehicles.groupby(["作業日期", "車種"]).size().reset_index(name="vehicles_needed")

# 75th percentile vehicle need per day
fleet_need_75 = daily_vehicle_use.groupby("車種")["vehicles_needed"].mean().reset_index()
fleet_need_75.columns = ["車種", "vehicles_needed_75th"]

print(fleet_need_75)


      車種  vehicles_needed_75th
0   1.9T              6.646341
1     機車             14.566265
2  電動車常溫              3.853659
